In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
DATA_IN_PATH = './data/' 
DATA_OUT_PATH = './output/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [3]:
train_data = pd.read_csv( DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [4]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

## TF-IDF 벡터화

In [5]:
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer='char', sublinear_tf=True, ngram_range=(1,3), max_features=5000)
# min_df = 설정한 값보다 특정 토큰의 df 값이 더 적게 나오면 벡터화 과정에서 제거
# analyzer = 분석하기 위한 기준 단위 word & char(word는 단어 단위, char = 문자 단위)
# sublinear_tf = 문서의 단어 빈도 수(term frequency)에 대한 스무딩(smoothing) 여부 설정
# ngram_range = 빈도의 기본 단위를 어느 범위의 n-gram으로 설정할 것인지
# max_features = 벡터의 최대 길이

In [6]:
X = vectorizer.fit_transform(reviews) # 전체 문장에 대한 특징 벡터 데이터

In [7]:
y = np.array(sentiments) # 정답 라벨을 넘파이 배열로 만듦

In [8]:
X

<25000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 17862871 stored elements in Compressed Sparse Row format>

In [9]:
features = vectorizer.feature_names() # reviews를 fit_transform 한 결과 나온 단어 목록

In [10]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)
# test_size = 원 데이터에서 훈련 검증 데이터를 나누는 비율 설정

## 로지스틱 회귀모형
### 학습 및 검증

In [11]:
lgs = LogisticRegression(class_weight = 'balanced')
lgs.fit(X_train, y_train)
# class_weight = balanced 는 각 라벨에 대해 균형있게 학습하도록 하는 인자값

c:\python\python36\venv\kmu\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [14]:
predicted = lgs.predict(X_eval) # 학습 시킨 모델에 예측 진행

In [16]:
print("Accuracy: %f" % lgs.score(X_eval, y_eval))
# lgs.score(X_eval, y_eval) = 검증 데이터로 성능 측정

Accuracy: 0.859600


## 로지스틱 회귀모형
### 테스트

In [17]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [19]:
testDataVecs = vectorizer.transform(test_data['review'])
# 테스트의 경우 fit_transform 이 아닌 transform으로 변환한다.

In [20]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted) # 예측값을 보면 긍정 또는 부정을 나타내는 0과 1로 결과값이 나온다.

[1 0 1 ... 0 1 0]


## 결과물 제출용 파일 저장

In [21]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset = pd.DataFrame({'id': test_data['id'], 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', index=False, quoting=3)

### 캐글에 데이터 제출시 결과 점수 : 0.85368
### public 점수로 비교해본 결과 314등정도(private 점수는 공개되지 않아 비교 불가)